# Numerical integration
Click [here](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fberkeley-physics%2FPython-Tutorials&urlpath=tree%2FPython-Tutorials%2F3+-+Specific+topics%2FNumerical+integration.ipynb&branch=master) to open this notebook in the DataHub.

## Learning objectives
By the end of this tutorial, you will be able to:
- Numerically integrate a function
- Numerically solve an inital-value problem

## Relevant documentation
- [Scipy `integrate` module](https://docs.scipy.org/doc/scipy/reference/integrate.html)

## Integrating functions
Suppose we wanted to integrate a function between two endpoints, i.e. evaluate $$\int_a^b f(x)\, dx.$$ Often we will encounter functions that are not easily integrable, in which case it might still be sueful to estimate the answer numerically. Of course, since we can only find one numerical value for each choice of parameter value at a time, analytic answers are always preferred.

The basic idea is to split the $[a,b]$ interval into many smaller subintervals, sample the function in each interval, and sum them appropriately. In the limit of infinitesimal subintervals, we recover the exact integral, but in practice, we keep increasing the number of intervals until the change in the answer is negligible. (Various transformations can be used if the interval is infinite, to map it to a finite range.)

The most convenient function for this is the `quad` function ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad)) in SciPy's `integrate` module, which automatically chooses which integration method to use based on the problem, and provides reliable results in a large variety of cases. Let's use it to evaluate $\int_0^1 e^{-x^2}\, dx$.

In [ ]:
import numpy as np
from scipy import integrate

def f(x):
    return np.exp(-x**2)

integrate.quad(f,0,1)

The returned tuple contains the answer first, and the estimated error second. Try using your own function, and/or use different bounds. (To take a limit to infinity for the bounds, use `np.inf` or `-np.inf`.) You might have seen before that $\int_{-\infty}^\infty e^{-x^2}\,dx=\sqrt{\pi}$. Is this consistent with the numerics?

There are several additional integration methods you can use which give you more control over the procedure, but which require more knowledge about what's going on under the hood. There are also options for evaluating an integral given only a fixed number of samples (rather than an evaluatable function). 

In higher dimensions, numerical integrals take longer, for obvious reasons. It might be advisable to use the `nquad` function ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.nquad.html#scipy.integrate.nquad)) rather than nest `quad` calls inside each other.

## Solving ODEs
This is a common problem in physics, where we want to solve for $x(t)$, given differential equation $$\dot x(t)=f(x(t),t),$$ and an initial value $x(0)$. In physics, we usually have second order differential equations $\ddot x(t)=f(x,t)$, but these can be made into first-order equations by doubling the dimension, defining the state vector $y=(x,\dot x)^T$, so $$\dot y(t) = \begin{pmatrix} \dot x(t)\\ \ddot x(t)\end{pmatrix}=\begin{pmatrix}y_2(t)\\ f(y_1(t),t)\end{pmatrix}=g(y(t),t).$$

The strategy for solving these is essentially the same: we split up the continuous time interval into many small intervals, and solve a discrete approximation to the problem at each interval. Since $x(t)$ can now accumulate error, it can be trickier to make sure that the errors have not propagated and that our answer makes sense.

The function `solve_ivp` ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp)) implements several algorithms that do this. Some methods are better for tricky problems, while the default method works best for most problems. It automatically estimates the error, and uses small enough step sizes that the error stays within the allowed bounds (that can be set using `atol` and `rtol`). 

You need to pass it a function which accepts the time $t$ first, and the state vector $y$ second, returning $\dot y$, as well as the initial value $y(t_0)$ and the time to which to integrate to, as a tuple $(t_0,t_f)$. It returns an object containing the time steps used by the solver, and the solution values $x(t)$ at these time steps, among other things. If you'd like $x(t)$ at specific times given by an array `times`, you can pass these times to the function using `t_eval=times`. You can also set `dense_output=True`, which tells it to also return a function representing $x$ (interpolated between the steps), which you can then call to obtain $x(t)$ at any $t$ you'd like.

The following cell integrates simple harmonic motion, $$ \frac{d}{dt}\begin{pmatrix}x\\\dot x\end{pmatrix}=\begin{pmatrix}\dot x\\-x\end{pmatrix}.$$

In [ ]:
def sho(t,y):
    return [y[1], -y[0]]

initial = [1,0]
time_range = (0,20)
soln = integrate.solve_ivp(sho, time_range, initial, dense_output=True)
soln

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(soln.t, soln.y[0], label="numerical")
#plt.plot(soln.t, np.cos(soln.t), label="exact")
#plt.legend()
plt.xlabel("$t$")
plt.ylabel("$x(t)$")

In [ ]:
plt.plot(soln.t, soln.y[1], label="numerical")
#plt.plot(soln.t, -np.sin(soln.t), label="exact")
#plt.legend()
plt.xlabel("$t$")
plt.ylabel(r"$\dot x(t)$")

In [ ]:
plt.plot(soln.y[0], soln.y[1], label="numerical")
#plt.plot(np.cos(soln.t), -np.sin(soln.t), label="exact")
#plt.legend()
plt.xlabel("$x(t)$")
plt.ylabel(r"$\dot x(t)$")
plt.gca().set_aspect("equal") #set equal scaling on x and y axes

As you can see, the solver used very large time-steps and was still able to give us an accurate answer. We can find $x(t)$ at more times by using the interpolated solution, stored as a function-like object in the `sol` attribute of the returned object.

In [ ]:
t_mesh = np.linspace(*time_range, 1000) #higher-resolution time array
y_mesh = soln.sol(t_mesh)
plt.plot(t_mesh, y_mesh[0], label="numerical")
#plt.plot(t_mesh, np.cos(t_mesh), label="exact")
#plt.legend()
plt.xlabel("$t$")
plt.ylabel("$x(t)$")

In [ ]:
plt.plot(t_mesh, y_mesh[1], label="numerical")
#plt.plot(t_mesh, -np.sin(t_mesh), label="exact")
#plt.legend()
plt.xlabel("$t$")
plt.ylabel(r"$\dot x(t)$")

In [ ]:
plt.plot(y_mesh[0], y_mesh[1], label="numerical")
#plt.plot(np.cos(t_mesh), -np.sin(t_mesh), label="exact")
#plt.legend()
plt.xlabel("$x(t)$")
plt.ylabel(r"$\dot x(t)$")
plt.gca().set_aspect("equal")

Feel free to play around with the options here. Try passing `t_mesh` into the `t_eval` argument to the solver to obtain `y_mesh` in `soln.y`. Try reducing the error tolerance, and comparing the performance. Try using a different integration method (by passing one into the `method` argument).

Finally, try integrating an IVP corresponding to a test body in a 2D gravitational well, which (in the appropriate units) has the ODE $$\frac{d}{dt}\begin{pmatrix}x\\y\\\dot x\\\dot y\end{pmatrix}=\begin{pmatrix}\dot x\\\dot y\\-\frac{x}{x^2+y^2}\\-\frac{y}{x^2+y^2}\end{pmatrix}.$$ Try finding bound and unbound orbits. Are the orbits closed? How efficient is the solver?